In [1]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py, os, sys, glob, tqdm
import pickle as pkl
import dlab.sglx_analysis as sglx
import dlab.nwbtools as nwbt

from dlab.nwbtools import df_from_phy, load_phy_template, option234_positions, multi_load_unit_data, make_spike_secs

import pynwb
from hdmf.backends.hdf5.h5_utils import H5DataIO
from hdmf.utils import StrDataset

from datetime import datetime
from dateutil.tz import tzlocal
from pynwb import NWBFile

%matplotlib inline
%load_ext autoreload
%autoreload 2

warnings.simplefilter(action='ignore', category=UserWarning)

no Allen Institute NWB API. get this from http://stash.corp.alleninstitute.org/projects/INF/repos/ainwb/browse
no OpenEphys.py get this from https://github.com/open-ephys/analysis-tools


In [2]:
recording_folder = r'Z:\color_representation\C133'
stim_path = os.path.join(recording_folder,'experiment_data')
int_path = os.path.join(recording_folder,'intermediates')
experimenter = 'js'
experiment_description= 'for Juans color representation project'

In [3]:
stim_df = pd.read_json(glob.glob(os.path.join(int_path,'*all_stim_df.json'))[0])
stim_df.head()

,stim,condition,sequence,color,frame,times,ori,posX,posY,TF,Phase
0,matrix,luminance,binary,bw,"[[-1, 1, -1, -1, 1, -1, -1, -1, 1, 1, 1, 1, -1...",108.510200,NaN,NaN,NaN,NaN,NaN
1,matrix,luminance,binary,bw,"[[-1, -1, -1, -1, 1, -1, -1, -1, -1, 1, 1, 1, ...",108.551833,NaN,NaN,NaN,NaN,NaN
2,matrix,luminance,binary,bw,"[[-1, -1, 1, 1, -1, 1, -1, -1, 1, 1, -1, 1, 1,...",108.602133,NaN,NaN,NaN,NaN,NaN
3,matrix,luminance,binary,bw,"[[1, -1, 1, 1, 1, -1, -1, -1, -1, -1, 1, 1, -1...",108.652567,NaN,NaN,NaN,NaN,NaN
4,matrix,luminance,binary,bw,"[[-1, -1, 1, 1, -1, -1, 1, 1, -1, 1, 1, -1, 1,...",108.702467,NaN,NaN,NaN,NaN,NaN


In [4]:
#folder_paths
continuous = os.path.join(recording_folder,'2023-04-10_17-28-19_C133','Record Node 102\\experiment1\\recording1\\continuous')
probe_a = os.path.join(continuous,'Neuropix-PXI-108.ProbeA-AP')
probe_b = os.path.join(continuous,'Neuropix-PXI-108.ProbeB-AP')
probe_c = os.path.join(continuous,'Neuropix-PXI-108.ProbeC-AP')
probes = [probe_a,probe_b,probe_c]

# turn spike times from samples to seconds
for probe in probes:
    make_spike_secs(probe)

#make dataframes and concat
probe_depths = [3500,3500,3500] #innacurate depth. Forgot to write down real depths
units_df = multi_load_unit_data(continuous,probe_depths=probe_depths, spikes_filename = 'spike_secs.npy')
units_df.head()

,probe,unit_id,group,depth,xpos,zpos,no_spikes,KSlabel,KSamplitude,KScontamination,template,waveform_weights,amplitudes,times
0,A,0,mua,3344.803199,29.665199,3684.803199,32943,mua,299.1,100.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.1843750476837158, 1.1843750476837158], [0....","[18.225805282592773, 16.112804412841797, 12.31...","[77.17209121913021, 77.21807042390323, 78.0425..."
1,A,1,mua,3452.930012,9.913181,3792.930012,449,mua,253.8,100.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.2933588027954102, 1.2933588027954102], [0....","[11.333195686340332, 11.059131622314453, 10.90...","[81.84445796342905, 88.43611733005976, 100.917..."
2,A,2,mua,3394.026614,24.509403,3734.026614,36686,mua,239.4,100.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.1483055353164673, 1.1483055353164673], [1....","[12.496817588806152, 19.329984664916992, 12.37...","[77.15263847864844, 77.16191439949495, 77.1937..."
3,A,3,mua,3353.989284,27.933475,3693.989284,13758,mua,502.3,100.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.3128201961517334, 1.3128201961517334], [1....","[39.613162994384766, 20.367557525634766, 20.82...","[78.15531564893756, 80.68680294630504, 80.7050..."
4,A,4,mua,3364.281632,27.993340,3704.281632,27027,mua,339.4,100.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0], [0.387484073638916, 0.38748407363...","[11.852208137512207, 11.66757583618164, 15.723...","[77.17659571306646, 77.18713956553947, 77.1881..."


In [5]:
# units_df = pd.read_json(glob.glob(os.path.join(int_path,'*spiketimes.json'))[0])
# units_df.head()

In [6]:
nwbfile = NWBFile('C133_2023041', 
                  recording_folder, 
                  datetime.now(tzlocal()),
                  experimenter=experimenter,
                  lab='Denman Lab',
                  institution='University of Colorado',
                  experiment_description=experiment_description,
                  session_id=os.path.basename(recording_folder))

In [7]:
#Add Epochs
nwbfile.add_epoch(stim_df[stim_df.sequence=='binary'][stim_df.condition=='luminance'].times.values[0],
                  stim_df[stim_df.sequence=='binary'][stim_df.condition=='luminance'].times.values[-1],
                  'luminance_binary')

nwbfile.add_epoch(stim_df[stim_df.stim == 'sweep map'].times.values[0],
                  stim_df[stim_df.stim == 'sweep map'].times.values[-1],
                  'sweep_map')

nwbfile.add_epoch(stim_df[stim_df.stim == 'masked gratings'].times.values[0],
                  stim_df[stim_df.stim == 'masked gratings'].times.values[-1],
                  'masked_gratings')

# nwbfile.add_epoch(stim_df[stim_df.sequence=='binary'][stim_df.condition=='color'].times.values[0],
#                   stim_df[stim_df.sequence=='binary'][stim_df.condition=='color'].times.values[-1],
#                   'color_binary')

# nwbfile.add_epoch(stim_df[stim_df.sequence=='ternary'][stim_df.condition=='color'].times.values[0],
#                   stim_df[stim_df.sequence=='ternary'][stim_df.condition=='color'].times.values[-1],
#                   'color_ternary')

In [8]:
nwbfile.add_trial_column('stimulus', 'the stimulus category')
nwbfile.add_trial_column('condition', 'the condition on which the stimulus varies')
nwbfile.add_trial_column('sequence', 'binary or ternary')
nwbfile.add_trial_column('color', 'green, uv ,or bw')
nwbfile.add_trial_column('frame','the matrix frame presented in a trial')
nwbfile.add_trial_column('frame_no','the matrix frame presented in a trial')
nwbfile.add_trial_column('TF','the gratings temporal frequency')
nwbfile.add_trial_column('SF','the gratings spatial frequency')
nwbfile.add_trial_column('ori','the orientation of a gratings')
nwbfile.add_trial_column('PosX', 'x position of stimulus object')
nwbfile.add_trial_column('PosY', 'x position of stimulus object')

In [9]:
stimulus = 'matrix'
condition = 'luminance'
sequence = 'binary'
color = 'bw'
stim = stim_df[stim_df.sequence==sequence][stim_df.condition==condition].reset_index()
stim.frame_no = np.arange(0,stim.shape[0])
for i,trial_start in enumerate(stim.times.values):
    nwbfile.add_trial(start_time=trial_start, stop_time=trial_start+np.mean(np.diff(stim.times)),
                      stimulus = stimulus, condition=condition, sequence=sequence, color=color, frame_no = stim.frame_no[i], frame = np.array(stim.frame[i]),
                      TF=np.nan,SF=np.nan,ori=np.nan,PosX=np.nan,PosY=np.nan)

In [10]:
stimulus = 'sweep map'
condition = np.nan
sequence = np.nan
color = 'bw'

stim = stim_df[stim_df.stim == stimulus]
stim.frame_no = np.arange(0,stim.shape[0])
for i,trial_start in enumerate(stim.times.values):
    nwbfile.add_trial(start_time=trial_start, stop_time=trial_start+np.mean(np.diff(stim.times)),
                      stimulus = 'sweep_map', condition=condition, sequence=sequence, color = color, frame_no = stim.frame_no[i], frame = np.zeros((64,64)),
                      TF=np.nan, SF=np.nan, ori=int(stim.ori.values[i]), PosX=np.nan,PosY=np.nan)

In [11]:
stimulus = 'masked gratings'
condition = np.nan
sequence = np.nan
color = 'bw'

stim = stim_df[stim_df.stim == stimulus]
stim.frame_no = np.arange(0,stim.shape[0])
for i,trial_start in enumerate(stim.times.values):
    nwbfile.add_trial(start_time=trial_start, stop_time=trial_start+np.mean(np.diff(stim.times)),
                      stimulus = 'masked_gratings', condition=condition, sequence=sequence, color = color, frame_no = stim.frame_no[i], frame = np.zeros((64,64)),
                      TF=stim.TF.values[i], SF=0.1, ori=int(stim.ori.values[i]), PosX=stim.posX.values[i],PosY=stim.posY.values[i])

In [12]:
stimulus = 'matrix'
condition = 'color'
sequence = 'binary'
color = 'green'

stim = stim_df[stim_df.sequence==sequence][stim_df.condition==condition][stim_df.color==color].reset_index()
stim.frame_no = np.arange(0,stim.shape[0])
for i,trial_start in enumerate(stim.times.values):
    nwbfile.add_trial(start_time=trial_start, stop_time=trial_start+np.mean(np.diff(stim.times)),
                      stimulus = stimulus, condition=condition, sequence=sequence, color = color, frame_no = stim.frame_no[i], frame = np.array(stim.frame[i]),
                      TF=np.nan,SF=np.nan,ori=np.nan,PosX=np.nan,PosY=np.nan)

In [13]:
stimulus = 'matrix'
condition = 'color'
sequence = 'binary'
color = 'uv'

stim = stim_df[stim_df.sequence==sequence][stim_df.condition==condition][stim_df.color==color].reset_index()
stim.frame_no = np.arange(0,stim.shape[0])
for i,trial_start in enumerate(stim.times.values):
    nwbfile.add_trial(start_time=trial_start, stop_time=trial_start+np.mean(np.diff(stim.times)),
                      stimulus = stimulus, condition=condition, sequence=sequence, color = color, frame_no = stim.frame_no[i], frame = np.array(stim.frame[i]),
                      TF=np.nan,SF=np.nan,ori=np.nan,PosX=np.nan,PosY=np.nan)

In [14]:
stimulus = 'matrix'
condition = 'color'
sequence = 'ternary'
color = 'green'

stim = stim_df[stim_df.sequence==sequence][stim_df.condition==condition][stim_df.color==color].reset_index()
stim.frame_no = np.arange(0,stim.shape[0])
for i,trial_start in enumerate(stim.times.values):
    nwbfile.add_trial(start_time=trial_start, stop_time=trial_start+np.mean(np.diff(stim.times)),
                      stimulus = stimulus, condition=condition, sequence=sequence, color = color, frame_no = stim.frame_no[i], frame = np.array(stim.frame[i]),
                      TF=np.nan,SF=np.nan,ori=np.nan,PosX=np.nan,PosY=np.nan)

In [15]:
stimulus = 'matrix'
condition = 'color'
sequence = 'ternary'
color = 'uv'

stim = stim_df[stim_df.sequence==sequence][stim_df.condition==condition][stim_df.color==color].reset_index()
stim.frame_no = np.arange(0,stim.shape[0])
for i,trial_start in enumerate(stim.times.values):
    nwbfile.add_trial(start_time=trial_start, stop_time=trial_start+np.mean(np.diff(stim.times)),
                      stimulus = stimulus, condition=condition, sequence=sequence, color = color, frame_no = stim.frame_no[i], frame = np.array(stim.frame[i]),
                      TF=np.nan,SF=np.nan,ori=np.nan,PosX=np.nan,PosY=np.nan)

In [16]:
probeids = ['A','B','C'] 
device = nwbfile.create_device('DenmanLab_EphysRig1','3 probe rig')

for i, probe in enumerate(probeids):
    electrode_name = 'probe'+str(i)
    description = "Neuropixels1.0_"+probeids[i]
    location = "somewhere in the brain"

    electrode_group = nwbfile.create_electrode_group(electrode_name,
                                                     description=description,
                                                     location=location,
                                                     device=device)
    
    #add channels to each probe
    for ch in range(option234_positions.shape[0]):
        nwbfile.add_electrode(x=option234_positions[ch,0],y=0.,z=option234_positions[0,1],imp=0.0,location='none',filtering='high pass 300Hz',group=electrode_group)

In [17]:
electrode_table_region = nwbfile.create_electrode_table_region([0], 'the second electrode')

In [18]:
nwbfile.add_unit_column('probe', 'probe ID')
nwbfile.add_unit_column('unit_id','cluster ID from KS2')
nwbfile.add_unit_column('depth', 'the depth of this unit from zpos and insertion depth')
nwbfile.add_unit_column('xpos', 'the x position on probe')
nwbfile.add_unit_column('zpos', 'the z position on probe')
nwbfile.add_unit_column('template','Template at each channel across timepoints [nTimepoints,nTempChannels]')
nwbfile.add_unit_column('amplitudes','Amplitude of each spike over time',index=True)
nwbfile.add_unit_column('no_spikes', 'total number of spikes across recording')
nwbfile.add_unit_column('group', 'user label of good/mua')
nwbfile.add_unit_column('KS_label', 'Kilosort label')
nwbfile.add_unit_column('KS_amp', 'Kilosort amplitude')
nwbfile.add_unit_column('KS_contam', 'Kilosort ISI contamination')
# nwbfile.add_unit_column('quality_snMax', 'signal:noise max')
# nwbfile.add_unit_column('quality_snMean', 'signal:noise mean')
# nwbfile.add_unit_column('quality_isolation', 'isolation distance')
# nwbfile.add_unit_column('quality_FLDAdprime', 'the quality for the inference of this unit')
# nwbfile.add_unit_column('quality_isiViolations', 'inter-spike interval violations')
# nwbfile.add_unit_column('quality_lda', 'linear discriminant analysis')

In [19]:
for i,unit_row in units_df.iterrows():
    nwbfile.add_unit(probe=str(unit_row.probe),
                     id = int(unit_row.name), #built-in
                     unit_id = unit_row.unit_id,
                     spike_times = unit_row.times, #built-in
                     electrodes = np.where(np.array(unit_row.waveform_weights) > 0)[0], #built-in
                     depth = unit_row.depth,
                     xpos = unit_row.xpos,
                     zpos = unit_row.zpos,
                     template = unit_row.template,
#                      amplitudes = np.array((unit_row.amplitudes,np.zeros(len(unit_row.amplitudes)))),
                     amplitudes = np.array(unit_row.amplitudes,dtype='object'),
                     no_spikes = unit_row.no_spikes,
                     group = str(unit_row.group),
                     KS_label = str(unit_row.KSlabel),
                     KS_contam = unit_row.KScontamination,)
#                      quality_snMax = unit_row.sn_max,
#                      quality_snMean = unit_row.sn_mean,
#                      quality_isolation = unit_row.isolation_distance,
#                      quality_FLDAdprime = unit_row.FLDA_dprime,
#                      quality_isiViolations = unit_row.isi_violations,
#                      quality_lda = unit_row.lda

In [1]:
nwb_path = os.path.join(recording_folder,os.path.basename(recording_folder))+'.nwb'
with pynwb.NWBHDF5IO(nwb_path, 'w') as io:
    io.write(nwbfile)

NameError: name 'os' is not defined